<a href="https://colab.research.google.com/github/AlexXPZhu/XMUM-FYP-Code/blob/main/FYP_DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn accelerate -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi
# 检查是不是在gpu上面运行

Wed Dec  3 08:44:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import pandas as pd

## Here, you can change the name of file to explore different files.
# 导入数据
file_path = '/content/drive/MyDrive/FYP/merged_data.csv'
df = pd.read_csv(file_path)

# 看看长什么样子
display(df.head(10))

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
5,?,1
6,' and 1 = 0 ) union all,1
7,? or 1 = 1 --,1
8,x' and userid is NULL; --,1
9,x' and email is NULL; --,1


In [ ]:
# 检查是不是链接到了cuda
import torch
print(f"Using device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

Using device: cuda


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# 训练 / 验证 / 测试划分
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["Label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["Label"])

# 转换为 Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(train_dataset, val_dataset, test_dataset)

Dataset({
    features: ['Sentence', 'Label', '__index_level_0__'],
    num_rows: 39827
}) Dataset({
    features: ['Sentence', 'Label', '__index_level_0__'],
    num_rows: 8535
}) Dataset({
    features: ['Sentence', 'Label', '__index_level_0__'],
    num_rows: 8535
})


In [ ]:
from transformers import DistilBertTokenizerFast

MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(
        examples["Sentence"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

encoded_train = train_dataset.map(preprocess_function, batched=True)
encoded_val = val_dataset.map(preprocess_function, batched=True)
encoded_test = test_dataset.map(preprocess_function, batched=True)

# Rename the 'Label' column to 'labels' for the model to correctly identify them
encoded_train = encoded_train.rename_column("Label", "labels")
encoded_val = encoded_val.rename_column("Label", "labels")
encoded_test = encoded_test.rename_column("Label", "labels")

for ds in [encoded_train, encoded_val, encoded_test]:
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/39827 [00:00<?, ? examples/s]

Map:   0%|          | 0/8535 [00:00<?, ? examples/s]

Map:   0%|          | 0/8535 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2  # 二分类
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [ ]:
  from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zxp1279839620 (zxp1279839620-xiamen-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000200,0.012041,0.997188,0.996415
2,0.000100,0.007285,0.998360,0.997904
3,0.000000,0.008626,0.998594,0.998203


TrainOutput(global_step=7470, training_loss=0.008267932590027838, metrics={'train_runtime': 680.9522, 'train_samples_per_second': 175.462, 'train_steps_per_second': 10.97, 'total_flos': 3956834314704384.0, 'train_loss': 0.008267932590027838, 'epoch': 3.0})

In [ ]:
import wandb
import os
import shutil

# Ensure wandb is running
if wandb.run is None:
    raise RuntimeError("W&B run not active. Please call wandb.init() before logging artifacts.")

# 1. Define a specific path for the final artifact to avoid mixing with checkpoints
# This creates a subfolder 'final_model' inside your output directory
final_model_dir = os.path.join(training_args.output_dir, "final_model")

# 2. Save Model and Tokenizer to this specific clean directory
print(f"Saving model and tokenizer to {final_model_dir}...")
trainer.save_model(final_model_dir)
tokenizer.save_pretrained(final_model_dir)

# 3. Create and Log the Artifact
# Note: 'type' helps you filter in the W&B UI (e.g., 'model', 'dataset', 'result')
artifact = wandb.Artifact(
    name="distilbert_finetuned_final",
    type="model",
    description="Final fine-tuned DistilBERT model with tokenizer"
)

# Add only the contents of the final_model_dir
artifact.add_dir(final_model_dir)

print("Uploading artifact to Weights & Biases...")
wandb.log_artifact(artifact)

# 4. Sync and Close (Critical for scripts)
wandb.finish()

print(f"✅ 微调完成。Model and Tokenizer uploaded to W&B.")

Saving model and tokenizer to /content/drive/MyDrive/results/final_model...


wandb: Adding directory to artifact (/content/drive/MyDrive/results/final_model)... Done. 4.5s


Uploading artifact to Weights & Biases...


eval/accuracy,▁▇█▆
eval/f1,▁▇█▆
eval/loss,▆▁▂█
eval/runtime,▁▁▁█
eval/samples_per_second,███▁
eval/steps_per_second,███▁
train/epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇██
train/grad_norm,▃▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,█████▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
+1,...


✅ 微调完成。Model and Tokenizer uploaded to W&B.


In [ ]:
# 直接用 trainer.evaluate() 计算测试集指标
test_results = trainer.evaluate(encoded_test)
print("📊 测试集指标：")
print(test_results)


📊 测试集指标：
{'eval_loss': 0.013935408554971218, 'eval_accuracy': 0.9981253661394259, 'eval_f1': 0.9976047904191617, 'eval_runtime': 9.4281, 'eval_samples_per_second': 905.274, 'eval_steps_per_second': 28.32, 'epoch': 3.0}
